In [1]:
import json
import re
import time
from typing import List, Optional

import pandas as pd
import requests
from tqdm.auto import tqdm


In [2]:
# there is probably a better way to get this list, but I am too lazy to do HTML scraping today, sorry
rs_guitarists_list = [
    """250 - Andy Summers (The Police)""",
    """249 - Brittany Howard (Alabama Shakes/Thunderbitch/solo)""",
    """248 - Robbie Kreiger (The Doors)""",
    """247 - Ricky Wilson (The B-52s)""",
    """246 - Paul Simon""",
    """245 - Leslie West (Mountain)""",
    """244 - Edilio Paredes (from Dominican Republic, in the Bachata genre)""",
    """243 - Aaron Dessner and Bryce Dessner (from The National... + Bryce - Steve Reich, Ryuichi Sakamoto, Jonny Greenwood, and Kronos Quartet. Aaron - Taylor Swift)""",
    """242 - Lindsey Jordan (Snail Mail)""",
    """241 - Keith Urban""",
    """240 - Erin Smith (Bratmobile)""",
    """239 - Duane Eddy""",
    """238 - Doug Gillard (Guided by Voices/Cobra Verde/Death of Samantha/GEM)""",
    """237 - Jennifer Batten""",
    """236 - Greg Sage (The Wipers/solo)""",
    """235 - Laura Marling""",
    """234 - John McGeoch (Magazine/Siouxsie & The Banshees/PiL)""",
    """233 - H.E.R.""",
    """232 - David Williams (session player - Michael Jackson, Chaka Khan, Kenny Loggins)""",
    """231 - Etta Baker""",
    """230 - Gustavo Cerati (from Argentina... Soda Stereo/solo)""",
    """229 - Barbara Lynn ("You'll Lose a Good Thing")""",
    """228 - Steve Jones (Sex Pistols)""",
    """227 - Glenn Branca""",
    """226 - El Kempner (Palehound)""",
    """225 - Fred ‘Sonic’ Smith and Wayne Kramer (MC5)""",
    """224 - Marv Tarplin (Smokey Robinson and the Miracles/Smokey Robinson)""",
    """223 - Joseph Spence (from The Bahamas)""",
    """222 - Molly Tuttle""",
    """221 - James Blood Ulmer""",
    """220 - Courtney Barnett""",
    """219 - Glenn Tipton and K.K. Downing (Judas Priest)""",
    """218 - Lzzy Hale (Halestorm)""",
    """217 - Thomas McClary (Ther Commodores)""",
    """216 - Steve Hackett (Genesis/solo)""",
    """215 - Kurt Vile""",
    """214 - Keiji Haino (from Japan - Aihiyo/solo)""",
    """213 - Lucy Dacus""",
    """212 - José Feliciano""",
    """211 - Nick Zinner (Yeah Yeah Yeahs)""",
    """210 - Kaki King""",
    """209 - Gary Clark Jr.""",
    """208 - Amadou Bagayoko (from Mali - Amadou and Mariam)""",
    """207 - Justin Broadrick (Godflesh)""",
    """206 - Hugh McCracken (session heavyweight - Steely Dan, Roberta Flack, Paul McCartney, Paul Simon)""",
    """205 - Eric Johnson""",
    """204 - Lynn Taitt (reggae/rocksetady session pioneer - Desmond Dekker)""",
    """203 - Grant Green""",
    """202 - Vince Gill""",
    """201 - Garry ‘Diaper Man’ Shider (Parliament/Funkadelic, Bootsy's Rubber Band)""",
    """200 - Christone ‘Kingfish’ Ingram""",
    """199 - Bombino (from Niger)""",
    """198 - Jerry Reed""",
    """197 - Nuno Bettencourt""",
    """196 - James Taylor""",
    """195 - Greg Ginn (Black Flag)""",
    """194 - Brian Robertson and Scott Gorham (Thin Lizzy)""",
    """193 - David Lindley (Jackson Browne/Linda Rondstadt/solo)""",
    """192 - Lita Ford""",
    """191 - Chalmers Edward ‘Spanky’ Alford (D'Angelo, Mary J. Blige, Total, Rapheal Saadique)""",
    """190 - Lightnin’ Hopkins""",
    """189 - Jerry Cantrell (Alice In Chains)""",
    """188 - Marnie Stern""",
    """187 - Marc Ribot""",
    """186 - Steve Lukather""",
    """185 - Peggy Jones ("Lady Bo")""",
    """184 - Eldon Shamblin (Bob Wills and His Texas Playboys)""",
    """183 - Roy Buchanan""",
    """182 - Earl ‘Chinna’ Smith (Bob Marley, Dennis Brown, Bunny Wailer, Sugar Minott, Jacob Miller, Black Uhuru, Mighty Diamonds, Augustus Pablo, Gregory Isaacs)""",
    """181 - Larissa Strickland (Laughing Hyenas)""",
    """180 - Mike Campbell (Tom Petty & The Heartbreakers)""",
    """179 - Ernest Ranglin (ska/rocksteady/reggae pioneer)""",
    """178 - Skip James""",
    """177 - Rodrigo y Gabriela""",
    """176 - Sadie Dupuis (Speedy Ortiz)""",
    """175 - Rory Gallagher""",
    """174 - Marty Stuart""",
    """173 - Paul McCartney""",
    """172 - Chrissie Hynde""",
    """171 - D. Boon (The Minutemen)""",
    """170 - Phil Manzanera (Roxy Music)""",
    """169 - Jessie Mae Hemphill""",
    """168 - John Cipollina (Quicksilver Messenger Service)""",
    """167 - James Williamson (Iggy & The Stooges)""",
    """166 - Johnny Winter""",
    """165 - Rokia Traoré (from Mali)""",
    """164 - Dave Davies (The Kinks)""",
    """163 - Wah Wah Watson""",
    """162 - Rosinha de Valença (from Brazil)""",
    """161 - Tim Henson (Polyphia)""",
    """160 - Kim and Kelley Deal (The Breeders)""",
    """159 - John Lennon""",
    """158 - Johnny Thunders""",
    """157 - Pat Metheny""",
    """156 - Carl Perkins""",
    """155 - Yvette Young (Covet)""",
    """154 - Bill Frisell""",
    """153 - Otis Rush""",
    """152 - Ani DiFranco""",
    """151 - Pete Cosey (Miles Davis)""",
    """150 - Marissa Paternoster (Screaming Females/Noun)""",
    """149 - Ron Asheton (The Stooges)""",
    """148 - Ike Turner""",
    """147 - Memphis Minnie""",
    """146 - Mike Bloomfield""",
    """145 - Dickey Betts""",
    """144 - Odetta""",
    """143 - Ira Kaplan (Yo La Tengo)""",
    """142 - João Gilberto""",
    """141 - Fredrik Thordendal (Meshugga)""",
    """140 - Stephen Stills""",
    """139 - Susan Tedeschi""",
    """138 - Teenie Hodges (Al Green/Hi Rhythm Section)""",
    """137 - Liz Phair""",
    """136 - Joe Perry (Aerosmith)""",
    """135 - Roger McGuinn (The Byrds)""",
    """134 - Bob Mould (Hüsker Dü/Sugar/solo)""",
    """133 - Robert Cray""",
    """132 - Nils Lofgren""",
    """131 - Dimebag Darrell (Pantera/Damage Plan)""",
    """130 - Joe Walsh""",
    """129 - Nita Strauss (Alice Cooper/Demi Lovato)""",
    """128 - Bob Stinson (The Replacements)""",
    """127 - Steve Vai""",
    """126 - Kim Thayil (Soundgarden)""",
    """125 - Viv Albertine (The Slits)""",
    """124 - Mike McCready and Stone Gossard (Pearl Jam)""",
    """123 - Steve Howe (Yes, Asia)""",
    """122 - King Sunny Ade""",
    """121 - Dick Dale""",
    """120 - Warren Haynes (Gov't Mule/Allman Brothers)""",
    """119 - Donita Sparks (L7)""",
    """118 - Adrian Belew""",
    """117 - Albert Collins""",
    """116 - Ana da Silva""",
    """115 - Nels Cline""",
    """114 - Robert Quine (Richard Hell & The Voidoids/Lou Reed/Matthew Sweet)""",
    """113 - Allen Collins and Gary Rossington (Lynyrd Skynyrd)""",
    """112 - Rowland S. Howard (The Birthday Party/Crime And The City Solution/These Immortal Souls)""",
    """111 - Kelley Johnson (Girlschool)""",
    """110 - Lindsey Buckingham (Fleetwood Mac, solo)""",
    """109 - Mick Ronson (David Bowie, Spiders From Mars/Ian Hunter/solo)""",
    """108 - Merle Travis""",
    """107 - Clarence White (The Byrds)""",
    """106 - Peter Buck (R.E.M.)""",
    """105 - Slash""",
    """104 - Ali Farka Touré (from Mali)""",
    """103 - Nancy Wilson (Heart)""",
    """102 - Billy Gibbons (ZZ Top)""",
    """101 - John Fogerty""",
    """100 - Kerry King (Slayer)""",
    """99 - Tosin Abasi (Animals As Leaders)""",
    """98 - Link Wray""",
    """97 - Stephen Malkmus (Pavement/Stephen Malkmus & The Jicks)""",
    """96 - Mark Knopfler (Dire Strais/solo)""",
    """95 - Mary Timony (Helium/solo/Wild Flag/Ex Hex)""",
    """94 - Joe Satriani""",
    """93 - Leo Nocentelli (The Meters)""",
    """92 - Wata (Boris)""",
    """91 - Cat Coore (Third World)""",
    """90 - Mdou Moctar (from Niger)""",
    """89 - Lou Reed""",
    """88 - Kurt Cobain (Nirvana)""",
    """87 - Poison Ivy (The Cramps)""",
    """86 - Sonny Sharrock""",
    """85 - Larry Carlton""",
    """84 - Muddy Waters""",
    """83 - Adrian Smith and Dave Murray (Iron Maiden)""",
    """82 - Wes Montgomery""",
    """81 - Bert Jansch""",
    """80 - Derek Trucks (Tedeshi-Trucks Band/Allman Brothers)""",
    """79 - Ernie Isley (Isley Brothers)""",
    """78 - Charlie Christian""",
    """77 - Willie Nelson""",
    """76 - Joan Jett""",
    """75 - Ritchie Blackmore (Deep Purple/Rainbow)""",
    """74 - J Mascis (Dinosaur Jr.)""",
    """73 - Hubert Sumlin (Howlin' Wolf)""",
    """72 - John McLaughlin (Mahavishnu Orchestra/Miles Davis)""",
    """71 - Franco Luambo (from Republic of the Cong/Zaire)""",
    """70 - Django Reinhardt""",
    """69 - Robbie Robertson (The Band)""",
    """68 - Les Paul""",
    """67 - Kevin Shields (My Bloody Valentine)""",
    """66 - Ry Cooder""",
    """65 - T-Bone Walker""",
    """64 - Carrie Brownstein (Sleeter-Kinney)""",
    """63 - Richard Thompson""",
    """62 - Peter Green (Fleetwood Mac)""",
    """61 - John Mayer""",
    """60 - Scotty Moore (Elvis Presley)""",
    """59 - Robert Fripp""",
    """58 - Alex Lifeson (Rush)""",
    """57 - Thurston Moore and Lee Ranaldo (Sonic Youth)""",
    """56 - Johnny Marr (The Smiths)""",
    """55 - Mick Taylor (The Rolling Stones)""",
    """54 - Bonnie Raitt""",
    """53 - Trey Anastasio (Phish)""",
    """52 - John Lee Hooker""",
    """51 - Tom Verlaine (Television/solo)""",
    """50 - Elmore James""",
    """49 - PJ Harvey""",
    """48 - Curtis Mayfield""",
    """47 - The Edge (U2)""",
    """46 - Frank Zappa""",
    """45 - Steve Cropper""",
    """44 - Johnny Ramone (The Ramones)""",
    """43 - Jonny Greenwood and Ed O’Brien (Radiohead)""",
    """42 - Vernon Reid (Living Color)""",
    """41 - Bo Diddley""",
    """40 - John Fahey""",
    """39 - Chet Atkins""",
    """38 - Angus Young and Malcolm Young (AC/DC)""",
    """37 - Pete Townshend (The Who)""",
    """36 - Elizabeth Cotten""",
    """35 - Eric Clapton (Bluesbreakers/Cream/Blind Faith/Derek & The Dominoes/solo/COVID-19)""",
    """34 - Jerry Garcia (Grateful Dead)""",
    """33 - Brian May (Queen)""",
    """32 - Jack White (White Stripes/solo)""",
    """31 - George Harrison""",
    """30 - Neil Young""",
    """29 - Eddie Hazel (Parliament/Funkadelic)""",
    """28 - David Gilmour (Pink Floyd)""",
    """27 - Buddy Guy""",
    """26 - St. Vincent""",
    """25 - John Frusciante""",
    """24 - James Burton (Ricky Nelson/Elvis/Emmylou Harris)""",
    """23 - James Hetfield and Kirk Hammett (Metallica)""",
    """22 - Albert King""",
    """21 - Randy Rhoads (Ozzy Osbourne)""",
    """20 - Stevie Ray Vaughan""",
    """19 - Freddy King""",
    """18 - Tom Morello (Rage Against The Machine/Audioslave)""",
    """17 - Mother Maybelle Carter (Carter Family)""",
    """16 - Robert Johnson""",
    """15 - Keith Richards (The Rolling Stones)""",
    """14 - Prince""",
    """13 - Tony Iommi (Black Sabbath)""",
    """12 - Jimmy Nolen (James Brown)""",
    """11 - Carlos Santana""",
    """10 - Duane Allman (Allman Brothers/Derek and The Dominoes)""",
    """9 - Joni Mitchell""",
    """8 - B.B. King""",
    """7 - Nile Rodgers (Chic/Diana Ross/Sister Sledge/David Bowie)""",
    """6 - Sister Rosetta Tharpe""",
    """5 - Jeff Beck""",
    """4 - Eddie Van Halen""",
    """3 - Jimmy Page""",
    """2 - Chuck Berry""",
    """1 - Jimi Hendrix""",
]


In [3]:
len(rs_guitarists_list)


250

In [4]:
rs_guitarists_list = [re.sub(r'\((.*?)\)', '', x).strip().replace('‘', "'").split(' - ') for x in rs_guitarists_list]


In [5]:
rs_guitarists_df = pd.DataFrame(rs_guitarists_list, columns=['rollingStonesRank', 'name'])

rs_guitarists_df.shape


(250, 2)

In [6]:
rs_guitarists_df.head()


,rollingStonesRank,name
0,250,Andy Summers
1,249,Brittany Howard
2,248,Robbie Kreiger
3,247,Ricky Wilson
4,246,Paul Simon


----- 

In [7]:
chatgpt_guitarists_list = [
    """Jimi Hendrix""",
    """Eric Clapton""",
    """Jimmy Page""",
    """Keith Richards""",
    """Jeff Beck""",
    """B.B. King""",
    """Chuck Berry""",
    """Eddie Van Halen""",
    """Stevie Ray Vaughan""",
    """David Gilmour""",
    """Carlos Santana""",
    """Slash""",
    """Brian May""",
    """Kirk Hammett""",
    """Angus Young""",
    """Tony Iommi""",
    """John Petrucci""",
    """Yngwie Malmsteen""",
    """Joe Satriani""",
    """Steve Vai""",
    """Robert Johnson""",
    """Duane Allman""",
    """Pete Townshend""",
    """George Harrison""",
    """Neil Young""",
    """Les Paul""",
    """Chet Atkins""",
    """Frank Zappa""",
    """Ry Cooder""",
    """Johnny Ramone""",
    """Jack White""",
    """John Frusciante""",
    """Tom Morello""",
    """Mark Knopfler""",
    """Randy Rhoads""",
    """Mick Taylor""",
    """Prince""",
    """Buddy Guy""",
    """Derek Trucks""",
    """John Mayer""",
    """Al Di Meola""",
    """Paul Gilbert""",
    """Ritchie Blackmore""",
    """Muddy Waters""",
    """John Lee Hooker""",
    """Steve Howe""",
    """Joe Pass""",
    """Wes Montgomery""",
    """Peter Green""",
    """Michael Schenker""",
    """Adrian Smith""",
    """Dave Mustaine""",
    """Alex Lifeson""",
    """Dimebag Darrell""",
    """Gary Moore""",
    """Robin Trower""",
    """Robert Fripp""",
    """Allan Holdsworth""",
    """John McLaughlin""",
    """Larry Carlton""",
    """Kenny Burrell""",
    """Billy Gibbons""",
    """Roy Buchanan""",
    """J.J. Cale""",
    """James Hetfield""",
    """Joni Mitchell""",
    """Peter Frampton""",
    """Zakk Wylde""",
    """Nuno Bettencourt""",
    """Phil Keaggy""",
    """Mike McCready""",
    """Steve Morse""",
    """Johnny Winter""",
    """Jan Akkerman""",
    """Pat Metheny""",
    """Robert Cray""",
    """Hank Marvin""",
    """Buddy Holly""",
    """Bonnie Raitt""",
    """John Scofield""",
    """Vernon Reid""",
    """Warren Haynes""",
    """Bo Diddley""",
    """Keb' Mo'""",
    """Ted Nugent""",
    """Ritchie Kotzen""",
    """Albert Collins""",
    """Jeff Healey""",
    """Andy Summers""",
    """Mike Stern""",
    """Lou Reed""",
    """Chuck Schuldiner""",
    """Marty Friedman""",
    """Robben Ford""",
    """Grant Green""",
    """Joe Perry""",
    """Brad Paisley""",
    """Jerry Garcia""",
    """Scotty Moore""",
    """Dick Dale""",
    """Stephen Stills""",
    """Carlos Montoya""",
    """Frank Marino""",
    """John Squire""",
    """John Cipollina""",
    """Elmore James""",
    """Steve Lukather""",
    """Son House""",
    """George Benson""",
    """Freddie King""",
    """Earl Hooker""",
    """Albert King""",
    """Otis Rush""",
    """Rory Gallagher""",
    """Carl Perkins""",
    """Julian Bream""",
    """Duane Eddy""",
    """Tommy Emmanuel""",
    """Derek Bailey""",
    """Peter Buck""",
    """Richard Thompson""",
    """Mick Ronson""",
    """Lenny Breau""",
    """Larry Coryell""",
    """Paco de Lucía""",
    """Johnny Marr""",
    """Charo""",
    """Andrés Segovia""",
    """Mississippi John Hurt""",
    """Lightnin' Hopkins""",
    """Leo Kottke""",
    """Glenn Tipton""",
    """K.K. Downing""",
    """John Fahey""",
    """Dave Davies""",
    """Clarence White""",
    """Terry Kath""",
    """Eddie Hazel""",
    """Sonny Sharrock""",
    """Daniel Ash""",
    """Link Wray""",
    """Gary Clark Jr.""",
    """Matthew Bellamy""",
    """Thurston Moore""",
    """Lee Ranaldo""",
    """Marnie Stern""",
    """John 5""",
    """Billy Corgan""",
    """Tom Verlaine""",
    """Robert Randolph""",
    """Kurt Cobain""",
    """Kim Thayil""",
    """Dave Navarro""",
    """Josh Homme""",
    """Isaac Brock""",
    """Daron Malakian""",
    """Kevin Shields""",
    """Nick Zinner""",
    """Jonny Greenwood""",
    """Ed O'Brien""",
    """Adam Jones""",
    """Ben Weinman""",
    """Stephen Carpenter""",
    """Tosin Abasi""",
    """Buckethead""",
    """Paul Waggoner""",
    """Brent Hinds""",
    """Bill Kelliher""",
    """Chris Letchford""",
    """Mark Holcomb""",
    """Misha Mansoor""",
    """Tim Henson""",
    """Scott LePage""",
    """Javier Reyes""",
    """Sarah Longfield""",
    """Plini""",
    """Guthrie Govan""",
    """Eric Johnson""",
    """Joe Bonamassa""",
    """Kenny Wayne Shepherd""",
    """Jon Gomm""",
    """Andy McKee""",
    """Michael Angelo Batio""",
    """Jason Becker""",
    """Marty Schwartz""",
    """Synyster Gates""",
    """Paul Masvidal""",
    """Allen Collins""",
    """Gary Rossington""",
    """Tony Rice""",
    """Steve Hackett""",
    """Richard Lloyd""",
    """Ty Tabor""",
    """Orianthi""",
    """Alvin Lee""",
    """Rik Emmett""",
    """Neal Schon""",
    """Matthias Jabs""",
    """Uli Jon Roth""",
    """Adrian Belew""",
    """Akira Takasaki""",
    """Vinnie Moore""",
    """George Lynch""",
    """John Sykes""",
    """Reb Beach""",
    """Doug Aldrich""",
    """Jake E. Lee""",
    """Warren DeMartini""",
    """Nita Strauss""",
    """Tony MacAlpine""",
    """Jennifer Batten""",
    """Yvette Young""",
    """Angel Vivaldi""",
    """Brad Delson""",
    """Mike Einziger""",
    """Jim Root""",
    """Mick Thomson""",
    """Mark Tremonti""",
    """Susan Tedeschi""",
    """Joe Walsh""",
    """Don Felder""",
    """Bernie Leadon""",
    """Steve Cropper""",
    """Peter Tosh""",
    """Bob Marley""",
    """Eric Gales""",
    """Pat Martino""",
    """Herb Ellis""",
    """Charlie Christian""",
    """Django Reinhardt""",
    """Richie Havens""",
    """Larry LaLonde""",
    """Jonny Lang""",
    """Lita Ford""",
    """Tommy Bolin""",
    """Alvin Youngblood Hart""",
    """Robbie Krieger""",
    """Robert Lockwood Jr.""",
    """Coco Montoya""",
    """R.L. Burnside""",
    """Hubert Sumlin""",
    """Luther Allison""",
    """Roy Rogers""",
    """Son Seals""",
    """Sue Foley""",
    """Kaki King""",
    """Philip Sayce""",
    """Ana Popovic""",
    """Julian Lage""",
    """Emily Remler""",
]


In [8]:
len(set(chatgpt_guitarists_list))


250

In [9]:
chatgpt_guitarists_list = [re.sub(r'\((.*?)\)', '', x).strip().replace('‘', "'") for x in chatgpt_guitarists_list]


In [10]:
chatgpt_guitarists_df = pd.DataFrame(
    data=zip(chatgpt_guitarists_list, range(1, len(chatgpt_guitarists_list) + 1)),
    columns=['name', 'chatGPTRank'],
)


chatgpt_guitarists_df.shape


(250, 2)

In [11]:
chatgpt_guitarists_df.head()


,name,chatGPTRank
0,Jimi Hendrix,1
1,Eric Clapton,2
2,Jimmy Page,3
3,Keith Richards,4
4,Jeff Beck,5


----- 

In [12]:
df = pd.merge(rs_guitarists_df, chatgpt_guitarists_df, how='outer')


df.shape


(392, 3)

In [13]:
df.head()


,rollingStonesRank,name,chatGPTRank
0,250,Andy Summers,89.0
1,249,Brittany Howard,NaN
2,248,Robbie Kreiger,NaN
3,247,Ricky Wilson,NaN
4,246,Paul Simon,NaN


----- 

In [14]:
def get_bing_image_url(names: List[str], subscription_key: str) -> List[Optional[str]]:
    """
    Retrieve the first HTTPS image URL for each name in the given list using Bing Image Search.

    Parameters
    ----------
    names: list of str
        List of names for which to find images
    subscription_key: str
        Subscription key for Bing Image Search API

    Returns
    -------
    image_urls: list of str
        List of image URLs corresponding to each name. If no image is found or an error occurs,
        ``None`` is placed in the list instead of a URL

    """
    image_urls_list = []

    for name in tqdm(names):
        try:
            response = requests.get(
                url='https://api.bing.microsoft.com/v7.0/images/search',
                headers={'Ocp-Apim-Subscription-Key': subscription_key},
                params={
                    'q': f'{name} guitarist',
                    'count': 20,
                    'offset': 0,
                    'mkt': 'en-US',
                    'safeSearch': 'Moderate'
                },
            )
            response.raise_for_status()
            search_results = response.json()

            first_image_url = next(
                (
                    item['contentUrl']
                    for item in search_results['value']
                    if 'https:' in item['contentUrl']
                ),
                None,
            )

            if first_image_url:
                image_urls_list.append(first_image_url)
            else:
                print(f'No suitable image found for {name} :(')
                image_urls_list.append(None)
        except Exception as e:
            print(f'Error with {name}: {e}')
            image_urls_list.append(None)

        time.sleep(0.33)  # my free API key only gets me 3 requests per second :)

    return image_urls_list


In [15]:
subscription_key = 'XXX'  # fill this in with your own API key, not mine!
image_urls = get_bing_image_url(names=df['name'].tolist(), subscription_key=subscription_key)


  0%|          | 0/392 [00:00<?, ?it/s]

In [16]:
# if this fails, you'll have to manually fill in an image URL
assert image_urls.count(None) == 0


In [17]:
df['imageUrl'] = image_urls


----- 

In [18]:
df = df[['name', 'rollingStonesRank', 'chatGPTRank', 'imageUrl']]


In [19]:
df['rollingStonesRank'] = df['rollingStonesRank'].astype(pd.Int64Dtype())
df['chatGPTRank'] = df['chatGPTRank'].astype(pd.Int64Dtype())


In [20]:
df.head()


,name,rollingStonesRank,chatGPTRank,imageUrl
0,Andy Summers,250,89,https://i0.wp.com/www.bossus.com/blog/wp-conte...
1,Brittany Howard,249,<NA>,https://www.premierguitar.com/media-library/ey...
2,Robbie Kreiger,248,<NA>,https://www.rollingstone.com/wp-content/upload...
3,Ricky Wilson,247,<NA>,https://i.pinimg.com/originals/ee/fd/25/eefd25...
4,Paul Simon,246,<NA>,https://assets.smoothradio.com/2016/26/paul-si...


----- 

In [21]:
with open(file='../guitarists.json', mode='w') as fp:
    json.dump(
        obj=json.loads(df.to_json(orient='records')),
        fp=fp,
        ensure_ascii=False,
        indent=4,
    )


----- 